In [1]:
from math import log2
from module.conf import PROJECT_DIR



In [2]:
# Read CSV
def load_csv(filename):
    data = []
    with open(filename, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        for line in lines[1:]:  # Ignore header
            category, message = line.strip().split(',', 1)
            message = message.strip('"')  # remove ()
            data.append([category, message])
    return data

In [3]:
# Extract keywords in msg
def extract_keywords(data, top_n=20):
    word_freq = {}
    for _, message in data:
        words = message.lower().replace(',', ' ').replace('.', ' ').replace('!', ' ').split()
        for word in words:
            word_freq[word] = word_freq.get(word, 0) + 1

    # sort by frequency and get top N
    sorted_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
    keywords = [word for word, _ in sorted_words[:top_n]]
    return keywords

# Extract features from msg
def extract_features(message, keywords):
    message = message.lower()
    features = []
    for keyword in keywords:
        count = message.count(keyword)
        features.append(count)
    return features

# Change data to number
def preprocess_data(data, keywords):
    processed_data = []
    for category, message in data:
        features = extract_features(message, keywords)
        label = 1 if category == 'spam' else 0  # spam: 1, ham: 0
        processed_data.append(features + [label])
    return processed_data

In [4]:
# Calculate Entropy
def calculate_entropy(labels):
    n = len(labels)
    if n == 0:
        return 0
    unique_labels = set(labels)
    entropy = 0
    for label in unique_labels:
        p = labels.count(label) / n
        if p > 0:
            # entropy -= p * __import__('math').log2(p)
            entropy -= p * log2(p)
    return entropy

# Calculate Information Gain
def calculate_information_gain(data, feature_idx, threshold):
    labels = [row[-1] for row in data]
    total_entropy = calculate_entropy(labels)
    n = len(data)

    left = [row for row in data if row[feature_idx] <= threshold]
    right = [row for row in data if row[feature_idx] > threshold]

    left_entropy = calculate_entropy([row[-1] for row in left])
    right_entropy = calculate_entropy([row[-1] for row in right])

    weighted_entropy = (len(left) / n) * left_entropy + (len(right) / n) * right_entropy
    gain = total_entropy - weighted_entropy
    return gain, left, right

# find best feature and threshold
def find_best_split(data):
    best_gain = -1
    best_feature = None
    best_threshold = None
    best_left = None
    best_right = None

    n_features = len(data[0]) - 1  # Number of Features (trừ nhãn)

    for feature_idx in range(n_features):
        values = sorted(set(row[feature_idx] for row in data))
        for i in range(len(values) - 1):
            threshold = (values[i] + values[i + 1]) / 2
            gain, left, right = calculate_information_gain(data, feature_idx, threshold)
            if gain > best_gain:
                best_gain = gain
                best_feature = feature_idx
                best_threshold = threshold
                best_left = left
                best_right = right

    return best_feature, best_threshold, best_gain, best_left, best_right

In [5]:
# Tree Node
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

# build Decision tree
def build_tree(data, max_depth, depth=0):
    labels = [row[-1] for row in data]

    if len(set(labels)) == 1:  # Tất cả cùng nhãn
        return Node(value=labels[0])
    if depth >= max_depth:  # Đạt độ sâu tối đa
        majority_label = max(set(labels), key=labels.count)
        return Node(value=majority_label)

    feature, threshold, gain, left, right = find_best_split(data)

    if gain <= 0:  # Không còn gain
        majority_label = max(set(labels), key=labels.count)
        return Node(value=majority_label)

    left_node = build_tree(left, max_depth, depth + 1)
    right_node = build_tree(right, max_depth, depth + 1)

    return Node(feature=feature, threshold=threshold, left=left_node, right=right_node)

def predict(tree, sample):
    if tree.value is not None:
        return tree.value

    if sample[tree.feature] <= tree.threshold:
        return predict(tree.left, sample)
    else:
        return predict(tree.right, sample)

In [6]:
def train_test_split(data, test_size=0.2):
    n = len(data)
    n_test = int(n * test_size)
    test_indices = set(__import__('random').sample(range(n), n_test))
    train_data = [data[i] for i in range(n) if i not in test_indices]
    test_data = [data[i] for i in range(n) if i in test_indices]
    return train_data, test_data

In [7]:
def main():
    filename = f"{PROJECT_DIR}/data/basic/email/spam.csv"
    raw_data = load_csv(filename)
    print(f"Loaded {len(raw_data)} samples from {filename}")

    keywords = extract_keywords(raw_data, top_n=200)  # Lấy 20 từ khóa phổ biến
    print(f"Feature keywords: {keywords}")

    data = preprocess_data(raw_data, keywords)

    train_data, test_data = train_test_split(data, test_size=0.2)
    print(f"Train set: {len(train_data)} samples")
    print(f"Test set: {len(test_data)} samples")

    max_depth = 3
    tree = build_tree(train_data, max_depth)

    def print_tree(node, indent=""):
        if node.value is not None:
            print(f"{indent}Leaf: {int(node.value)}")
        else:
            print(f"{indent}Feature '{keywords[node.feature]}' count <= {node.threshold}")
            print_tree(node.left, indent + "  Left: ")
            print_tree(node.right, indent + "  Right: ")

    print("\nDecision Tree structure:")
    print_tree(tree)

    correct = 0
    for sample in test_data:
        pred = predict(tree, sample[:-1])
        if pred == sample[-1]:
            correct += 1
    accuracy = correct / len(test_data)
    print(f"\nTest Accuracy: {accuracy * 100:.2f}%")

    # Dự đoán một tin nhắn mới
    new_message = "Free tickets to win a prize! Call now!"
    new_sample = extract_features(new_message, keywords)
    prediction = predict(tree, new_sample)
    print(f"Predict new message: '{new_message}': {'spam' if prediction == 1 else 'ham'}")

if __name__ == "__main__":
    main()

Loaded 5574 samples from /Users/hiepnq/Working/training/python/learn-python/data/basic/email/spam.csv
Feature keywords: ['i', 'to', 'you', 'a', 'the', 'u', 'and', 'is', 'in', 'me', 'my', 'for', 'your', 'it', 'of', 'call', 'have', 'on', 'that', 'are', '2', 'now', 'so', 'but', 'not', 'or', 'can', 'at', 'do', 'will', "i'm", 'ur', 'be', 'if', 'get', 'with', 'just', 'we', 'this', 'no', 'up', 'when', 'from', '4', 'go', '&lt;#&gt;', 'ok', 'free', 'all', 'how', 'out', 'what', 'know', 'like', 'then', 'good', 'got', 'was', 'come', 'am', 'its', 'love', 'time', 'only', '?', 'day', 'send', 'he', 'there', 'want', 'text', 'as', 'by', 'one', "i'll", 'need', 'ü', 'home', 'going', 'about', 'lor', 'sorry', 'see', 'still', 'txt', 'r', 'n', 'reply', 'dont', 'back', 'our', 'she', 'stop', "don't", 'tell', 'mobile', 'new', 'take', 'hi', 'da', 'any', 'today', 'please', 'pls', 'think', 'been', 'they', 'her', 'later', 'k', 'did', 'dear', 'phone', 'some', 'has', 'well', 'great', 'an', 'hey', 'here', 'claim', 'hop